## 1. Load packages

In [ ]:
# data processing packages
import numpy as np
from numpy import random
import pandas as pd
import re

# machine learning packages
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.decomposition import PCA

# visualization packages
import seaborn as sb
import matplotlib.pyplot as plt

# other packages
import torch
from pathlib import Path
import random
import itertools

## 2. Read the files

In [ ]:
## Create the necessary folders
Path('./Figures/').mkdir(parents=True, exist_ok=True)
Path('./Results/').mkdir(parents=True, exist_ok=True)

In [ ]:
## Set path to the data set
dataset_path = "./dataset/77_cancer_proteomes_CPTAC_itraq.csv"
clinical_info = "./dataset/clinical_data_breast_cancer.csv"
pam50_proteins = "./dataset/PAM50_proteins.csv"

## Load data
data = pd.read_csv(dataset_path,header=0,index_col=0)
clinical_file = pd.read_csv(clinical_info,header=0,index_col=0) ## holds clinical information about each patient/sample
pam50 = pd.read_csv(pam50_proteins,header=0)

# RefSeq protein ID (each protein has a unique ID in a RefSeq database)
print(data.index.name)
data.head()

## 3. Data Set Processing

In [ ]:
## Drop unused information columns
data.drop(['gene_symbol','gene_name'],axis=1,inplace=True)

## Change the protein data sample names to a format matching the clinical data set
data.rename(columns=lambda x: "TCGA-%s" % (re.split('[_|-|.]',x)[0]) if bool(re.search("TCGA",x)) is True else x,inplace=True)

data.head()

In [ ]:
## Transpose data for the clustering algorithm since we want to divide patient samples, not proteins
print(data.shape)
datat = data.transpose()
print(datat.shape)

datat.head()

In [ ]:
print("Number of patients in clinical data set: ", len(clinical_file.index))
print("Number of patients in protein data set: ", len(datat.index))

In [ ]:
## Drop clinical entries for samples not in our protein data set
clinical = clinical_file.loc[[x for x in clinical_file.index.tolist() if x in datat.index],:]

print("The shape of the clinical data set: ", clinical.shape)
clinical.head()

In [ ]:
## Add clinical meta data to our protein data set, note: all numerical features for analysis start with NP_ or XP_
merged = datat.merge(clinical,left_index=True,right_index=True)

# Drop the duplicated columns (added by Pietro Gavazzi)
liste = merged.index.copy()
liste = list(liste)

for i in np.unique(merged.index):
    liste.remove(i)

## Change name to make it look nicer in the code!
processed = merged.drop(np.unique(liste))

print("Shape of the merged data set: ", processed.shape)
print("with %d patients and %d features" % (processed.shape[0], processed.shape[1]))

processed.head()

In [ ]:
## Numerical data for the algorithm, NP_xx/XP_xx are protein identifiers from RefSeq database
X = processed.loc[:,[x for x in processed.columns if bool(re.search("NP_|XP_|YP_",x)) == True]]
Y = pd.get_dummies(processed.drop(X.columns, axis=1)['Integrated Clusters (with PAM50)'], prefix="PAM50")

## Select only the PAM50 proteins - known panel of genes used for breast cancer subtype prediction
# processed_numerical_p50 = processed_numerical.iloc[:,processed_numerical.columns.isin(pam50['RefSeqProteinID'])]
# processed_numerical_p50.head()

X.columns

In [ ]:
Y.head()

In [ ]:
# Save the data
torch.save(X, './Results/X')
torch.save(Y, './Results/Y')

## 3. Data Engineering

In [ ]:
# Read the processed data set
X = torch.load('./Results/X')
Y = torch.load('./Results/Y')

In [ ]:
X.head()

In [ ]:
Y.head()

### 3.1 Missing value process
We drop all the columns with nan values

In [ ]:
nan_counts = np.sum(X.isna(), axis=0)
col_to_drop = nan_counts[nan_counts != 0].index
X.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
X.shape

### 3.2 Data Scaling
We use the standardization for each feature of the data set

In [ ]:
col_names = X.columns
idx = X.index
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=col_names, index=idx)
X_scaled.shape

## 4. Basic model

### 4.1 K-Means model

In [ ]:
kmeans = KMeans(n_clusters=4, algorithm='full').fit(X_scaled)
pred = kmeans.labels_

### 4.2 Evaluation Metric

In [ ]:
permutation_matrices = []
for i in itertools.permutations([0, 1, 2, 3]):
    matrix = np.zeros((4, 4))
    for j in range(len(matrix)):
        matrix[j][i[j]] = 1

    permutation_matrices.append(matrix)

def model_eval(y_true, y_pred, return_permutation=False, plot=False):
    pred = pd.DataFrame(pd.get_dummies(y_pred))
    pred.columns = ['cluster_' + str(x) for x in range(1, 5)]
    pred = pred.set_index(y_true.index)
    merged = y_true.merge(pred, right_index=True, left_index=True)

    matrix = np.zeros((len(merged.T), len(merged.T)))

    indi = 0
    for i in merged.T.index:
        indj = 0
        for j in merged.T.index:
            matrix[indi][indj] += np.array(merged[i]) @ np.array(merged[j])
            indj+=1
        indi+=1

    for i in range(len(matrix)):
        matrix[i] /= matrix[i][i]

    # # diag = np.diag(matrix)
    # matrix /= merged.shape[0]

    # file_name = './Figures/' + name + '.pdf'
    #
    # plt.figure(figsize=(6, 6), dpi=100)
    # ax = sb.heatmap(matrix, annot=True, cbar=False)
    # ax.set_xticklabels(merged.columns, rotation=45)
    # ax.set_yticklabels(merged.columns, rotation=0)
    # plt.title(name)
    # plt.savefig(file_name)
    # plt.show()

    ## version 1.0
    # cm = matrix[4:, :4]
    # accuracy_max = -np.Inf
    # best_comb = None
    # for comb in itertools.permutations([0, 1, 2, 3]):
    #     acc = 0
    #     i = 0
    #     for j in comb:
    #         acc += cm[i, j]
    #         i += 1
    #
    #     if acc > accuracy_max:
    #         accuracy_max = acc
    #         best_comb = comb
    #
    # cm_with_order = np.concatenate([cm, np.array(best_comb).reshape(-1, 1)], axis=1)
    # new_cm = cm_with_order[cm_with_order[:, 4].argsort()][:, :4]
    #
    # if plot:
    #     plt.figure(figsize=(6, 6), dpi=100)
    #     ax = sb.heatmap(new_cm, annot=True, cbar=False)
    #     ax.set_yticklabels(['cluster_' + str(x) for x in range(1, 5)], rotation=0)
    #     ax.set_xticklabels(['PAM50_' + str(x) for x in range(1, 5)], rotation=45)
    #     plt.show()

    # return accuracy_max


    upper_mat = matrix[:4, 4:]
    lower_mat = matrix[4:, :4]


    best_upper_dist, best_lower_dist = np.Inf, np.Inf
    best_upper_permu, best_lower_permu = None, None

    ## version 2.0
    for permu_mat in permutation_matrices:
        upper_dist = np.sum(np.abs(permu_mat - upper_mat))
        if upper_dist < best_upper_dist:
            best_upper_dist = upper_dist
            best_upper_permu = permu_mat

        lower_dist = np.sum(np.abs(permu_mat - lower_mat.T))
        if lower_dist < best_lower_dist:
            best_lower_dist = lower_dist
            best_lower_permu = permu_mat

    error = best_upper_dist + best_lower_dist

    lower_mat =  best_lower_permu @ lower_mat
    new_matrix = np.hstack([np.vstack([matrix[:4, :4], lower_mat]), np.vstack([upper_mat, matrix[4:, 4:]])])

    if plot:
        plt.figure(figsize=(6, 6), dpi=100)
        ax = sb.heatmap(new_matrix, annot=True, cbar=False)
        ax.set_xticklabels(merged.columns, rotation=45)
        ax.set_yticklabels(merged.columns, rotation=0)
        plt.show()

    if return_permutation:
        return error, best_upper_permu, best_lower_permu
    else:
        return error

In [ ]:
error = model_eval(Y, pred)

In [ ]:
error

## 5. Model improvement

### 5.1 Feature selection

In [ ]:
error_list = []
min_error = np.Inf
feature_selected = None
for i in range(X_scaled.shape[1]):

    X_reduced = np.array(X_scaled.iloc[:, i]).reshape(-1, 1)

    k_means = KMeans(n_clusters=4, algorithm='full').fit(X_reduced)
    pred_ = k_means.labels_
    error = model_eval(Y, pred_)
    error_list.append(error)
    if error < min_error:
        min_error = error
        feature_selected = X_scaled.columns[i]

print(min_error)
print(feature_selected)

In [ ]:
X_reduced = np.array(X_scaled[feature_selected]).reshape(-1, 1)
k_means = KMeans(n_clusters=4, algorithm='full').fit(X_reduced)
pred = k_means.labels_
error = model_eval(Y, pred, False, True)

### 5.2 Model selection